In [1]:
import pandas as pd
import numpy as np
import os
import sys
import requests
import re
from datetime import datetime

In [2]:
class Preprocesser:
    def __init__(self):
        self.kospi = ""
        self.kosdaq = ""
        # need kospi file
        # MK Embedding file
    
        #papago 번역 함수
    
    
    #폴더에 있는 파일 리스트 가져오기
    def get_filelist(self):
        filelist = os.listdir(os.getcwd())
        return filelist


    #파일 불러오기
    def load_funda_data(self, data_name):
        data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')
        return data    

    def load_feature_data(self, data_name):
        if 'csv' in data_name:
            data = pd.read_csv("{}".format(data_name), thousands=',')
        else:
            data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')


    # 재무 데이터 컬럼 이름 설정
    def set_col_names(self, df):
        col_list = ['Date', 'asset(1000)', 'debt(1000)', 'sales(1000)',
                    'adjusted_price', 'profit_rate', 'volumne(mean_5d)(week)',
                    'volumne(week)','profit(1000)',
                    'term_profit(1000)', 'gross_margin(1000)','adjusted_BPS', 'adjusted_EPS']
        df.columns = col_list
        return df
    
    def add_feature_name(self, feature_name_list, file_name):
        dotpos = file_name.find('.')
        file_name = file_name[:dotpos]
        new_col = ['Date']
        for i in range(feature_name_list)-1:
            new_name = feature_name_list[i+1]+file_name
            new_col += new_name
        return new_col


#     # 피쳐 데이터 컬럼 이름 설정(2차전지) (나머지 산업에 대해서도 필요)
#     def feature_col_name(df, name):
#         """
#         name : dubai, nickel, mirenal, wti, lithum, cobalt_l,cobalt_r, brent

#         """
#         if name =='dubai':
#             col_list = dubai_price_col_list = ['Date', 'dubai_price','dubai_price_change','dubai_price_change_rate','dubai_stock']
#         elif name == 'nickel':
#             col_list = nickel_price_col_list = ['Date', 'nickel_price','nickel_price_change','nickel_price_change_rate','nikcel_stock']
#         elif name == 'mineral':
#             col_list = mineral_col_list = ['Date', 'mineral_index','mineral_index_change','mineral_index_change_rate']
#         elif name == 'wti': 
#             col_list = wti_price_col_list = ['Date', 'wti_price','wti_price_change','wti_price_change_rate','wti_stock']
#         elif name == 'lithum':
#             col_list = lithum_price_col_list = ['Date', 'lithum_price','lithum_price_change','lithum_price_change_rate','lithum_stock']
#         elif name == 'cobalt_l': 
#             col_list = cobalt_l_price_col_list = ['Date', 'cobalt_l_price','coblat_l_price_change','cobalt_l_price_change_rate','cobalt_l_stock']
#         elif name == 'cobalt_r': 
#             col_list = cobalt_r_price_col_list = ['Date', 'cobalt_r_price','coblat_r_price_change','cobalt_r_price_change_rate','cobalt_r_stock']
#         elif name == 'brent': 
#             col_list = brent_price_col_list = ['Date', 'brent_price','brent_price_change','brent_price_change_rate','brent_stock']
        
#         df.columns = col_list

#         return df


    # 인덱스 설정하고 원하는 연도부터 프레임 설정(재무)
    def set_funda_data(self, data, start_date):

        """
        set_funda_data(df,'2010-01-05')
        """
        data = data.iloc[5:]

        data.Date =  pd.to_datetime(data.Date)
        data.set_index(['Date'],inplace=True)
        data = data.loc[start_date:]
        return data


    # 인덱스 설정하고 원하는 연도부터 프레임 설정(피쳐)
    def set_feature_data(self, data, start_date):

        """
        set_funda_data('samsung_elec",'2010-01-05')
        """
        
        data.Date =  pd.to_datetime(data.Date)
        data.set_index(['Date'],inplace=True)
        data = data.loc[start_date:]
        return data



    #화장품 fundamental data 내 존재하는 monthly data를 날짜별로 만들기  ( 현재 datetime으로 되어 있고, 모든 날짜에 값 부여 필요 )
    def mkdatetime_mth(self, data):
        data2 = data[data['년월']['년월'].str.contains("월")]
        index = data2.index
        for i, mth in enumerate(data2['년월']['년월']):
            m = int(mth[1:-1])
            idx = index[i]
            idx = idx - m
            y = int(data['년월']['년월'][idx][:-1])
            print(y, m)

            data2.iat[i, 0] = datetime(y, m, 1)

        return data2






    # #데이터 불러와서 datetime으로 만들고, 각 컬럼 모두 합치기, 수연이한테 각 컬럼명이 무엇을 뜻하는 지 물어볼 것
    # def read_mth_data(self, data, domain):
    #     domain = "화장품"
    #     os.chdir(r"G:\공유 드라이브\Boad ADV Stock\{}\fundamental".format(domain))
        
    #     data = pd.read_excel("330112_오렌지유.xls", skiprows = [0, 1], header=[0,1])
    #     pd.read_excel("330113_레몬유.xls", skiprows = [0, 1], header=[0,1])



    # 특정 연도 사이 휴일 리스트
    def check_holiday(self, start_year, last_year):
        
        """
        check_hodiday(2010,2020)
        """

        total_holiday = list()
        for year in range(start_year, last_year):
            hlist = list()
            krholiday = [d.strftime('%Y-%m-%d') for d in pytimekr.holidays(year)]
            week_sat = [d.strftime('%Y-%m-%d') for d in pd.date_range('{}-01-01'.format(year),'{}-12-31'.format(year),freq='W-SAT')]
            week_sun = [d.strftime('%Y-%m-%d') for d in pd.date_range('{}-01-01'.format(year),'{}-12-31'.format(year),freq='W-SUN')]
            hlist.extend(krholiday)
            hlist.extend(week_sat)
            hlist.extend(week_sun)
            total_holiday.extend(pd.unique(hlist))
        sorted(total_holiday)
        return total_holiday


    # 특정 연도 사이 휴일 x 리스트
    def check_workday(self, start_year, last_year):
        """
        check_workday(2010,2020)
        """

        workday = list()
        for d in pd.date_range('{}-01-01'.format(start_year),'{}-12-31'.format(last_year)):
            if d.strftime('%Y-%m-%d') not in check_holiday(start_year, last_year):
                workday.append(d.strftime('%Y-%m-%d'))       
        return workday


    # 데이터 프레임에서 휴일 아닌 날만 뽑기
    def set_index_workday(self, df,start_year,last_year):
        work = pd.DataFrame(check_workday(start_year, last_year),columns=['Date'])
        work.Date =  pd.to_datetime(work.Date)
        df.reset_index(inplace=True)
        work_df = pd.merge(df,work, on ='Date',how='inner')
        work_df.set_index(['Date'],inplace=True)
        # work_df = pd.concat([df,work], join_axes=['Date'],join='inner',axis=0)

        return work_df

    # 회시별 재무 데이터 테이블
    def individual_feauture(self, stockname):
        df = load_funda_data(stockname)
        df = set_col_names(df)
        df = set_funda_data(df)
        return df

    # 산업 공통 피쳐
    def common_feature(self,featurename):
        
        df = load_funda_data(featurename)
        df = feature_col_name(df)
        df = set_feature_data(df)
        return df

    # 산업 공통 피쳐 테이블
    def common_feature_table(self):
        os.chdir(r'G:\공유 드라이브\Boad ADV Stock\2차전지\funda_data')
        filelist = os.listdir(os.getcwd())
        # filelist
        feature_df = pd.DataFrame()
        for feature in filelist:
            add_df = common_feature(feature)
            feature_df = pd.merge(feature_df, add_df, on='Date', how ='left')
        return


In [3]:
class livestock_preprocesser(Preprocesser):
    
    def get_filelist(self, folder):
        #folder = 'fn_data' , 'tech_data'
        filelist = os.listdir(os.chdir(r'G:\공유 드라이브\Boad ADV Stock\축산업\{}'.format(folder)))
        return filelist

In [4]:
# os.getcwd(r'G:\공유 드라이브\Boad ADV Stock\축산업')

In [5]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\축산업')

In [6]:
os.getcwd()

'G:\\공유 드라이브\\Boad ADV Stock\\축산업'

In [7]:
os.chdir('./fn_data')

In [8]:
os.getcwd()

'G:\\공유 드라이브\\Boad ADV Stock\\축산업\\fn_data'

In [9]:
livestock_preprocesser = livestock_preprocesser()

In [10]:
livestock_preprocesser.get_filelist('fn_data')

['이지바이오.xlsx', '대주산업.xlsx', '케이씨피드.xlsx', '한일사료.xlsx']

In [11]:
# data  = livestock_preprocesser.load_funda_data(livestock_preprocesser.get_filelist('fn_data')[3])
company = '이지바이오'
data = pd.read_excel('{}.xlsx'.format(company))

In [12]:
# data.head(8)

In [13]:
data = livestock_preprocesser.set_col_names(data)
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,Symbol Name,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오,이지바이오
1,Kind,NFS-IFRS(C),NFS-IFRS(C),NFS-IFRS(C),SSC,SSC,SSC,SSC,NFS-IFRS(C),NFS-IFRS(C),NFS-IFRS(C),NVAL-IFRS(C),NVAL-IFRS(C)
2,Item,6000901001,6000902001,6000904001,S410000700,S41000170F,S41000620F,S41000080F,6000906001,6000908001,6000904007,6000601036,6000601003
3,Item Name,총자산(천원),총부채(천원),매출액(천원),수정주가(원),수익률(%),거래량 (5일 평균)(주),거래량(주),영업이익(천원),당기순이익(천원),매출총이익(천원),수정BPS(원),수정EPS(원)
4,Frequency,1Q,1Q,1Q,DAILY,DAILY,DAILY,DAILY,1Q,1Q,1Q,1Q,1Q


In [14]:
data = livestock_preprocesser.set_funda_data(data , '2010-01-11')
data.head()

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
Date,,,,,,,,,,,,
2010-01-11,NaN,NaN,NaN,2450,-3.73,2054718,1186071,NaN,NaN,NaN,NaN,NaN
2010-01-12,NaN,NaN,NaN,2640,7.76,2443731,2551680,NaN,NaN,NaN,NaN,NaN
2010-01-13,NaN,NaN,NaN,2575,-2.46,1879920,2561843,NaN,NaN,NaN,NaN,NaN
2010-01-14,NaN,NaN,NaN,2545,-1.17,1670983,1032237,NaN,NaN,NaN,NaN,NaN
2010-01-15,NaN,NaN,NaN,2660,4.52,2226197,3799154,NaN,NaN,NaN,NaN,NaN


In [15]:
data = data.reset_index()
data.replace('-',np.nan, inplace=True)
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,2010-01-11,NaN,NaN,NaN,2450,-3.73,2054718,1186071,NaN,NaN,NaN,NaN,NaN
1,2010-01-12,NaN,NaN,NaN,2640,7.76,2443731,2551680,NaN,NaN,NaN,NaN,NaN
2,2010-01-13,NaN,NaN,NaN,2575,-2.46,1879920,2561843,NaN,NaN,NaN,NaN,NaN
3,2010-01-14,NaN,NaN,NaN,2545,-1.17,1670983,1032237,NaN,NaN,NaN,NaN,NaN
4,2010-01-15,NaN,NaN,NaN,2660,4.52,2226197,3799154,NaN,NaN,NaN,NaN,NaN


In [16]:
livestock_preprocesser.get_filelist('feature_data')

# ['livecattle_f.csv',
##  '육계.xlsx',
##  '육계_도매.xlsx',
#  'bdi.xlsx',
#  'KOSPI Volatility.csv',
#  'South Korea 5-Year Bond Yield.csv',
#  'USD_KRW.csv',
#  'grain.xlsx',
#  'leanhog_f.csv',
#  '~$육계.xlsx']

['bdi.xlsx',
 'livecattle_f.csv',
 'South Korea 5-Year Bond Yield.csv',
 'USD_KRW.csv',
 'leanhog_f.csv',
 'kospi_volatility.csv',
 'grain.xlsx',
 'zNex~$hareSouth Korea 5-Year Bond Yield.csv',
 'rl_data.csv',
 'rl_data_for_test.csv',
 'kospi,kosdaq.xlsx']

# livecattle_f

In [17]:
# data  = livestock_preprocesser.load_funda_data(livestock_preprocesser.get_filelist('feature_data')[0])
# data.head()
data_name = livestock_preprocesser.get_filelist('feature_data')[1]

In [18]:
data_name

'livecattle_f.csv'

In [19]:
# data = pd.read_excel("{}".format(data_name),encoding='cp949', thousands=',')
data1 = pd.read_csv("{}".format(data_name), thousands=',')

In [20]:
data1.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 31일,124.700,124.400,124.750,124.400,0.34K,0.48%
1,2019년 12월 30일,124.100,123.650,124.125,123.575,0.79K,0.49%
2,2019년 12월 27일,123.500,123.300,123.900,123.200,1.01K,0.18%
3,2019년 12월 26일,123.275,122.650,123.275,122.550,1.04K,-1.99%
4,2019년 12월 25일,125.775,125.775,125.775,125.775,-,2.55%


In [21]:
# livecattle_f
col_list = ['Date','close','open','high','low','volume','change%']

In [22]:
data1.columns = col_list

In [23]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name[:dotpos]
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

In [24]:
data1.columns = add_feature_name(col_list,data_name)

In [25]:
data1.head(10)

,Date,close_livecattle_f,open_livecattle_f,high_livecattle_f,low_livecattle_f,volume_livecattle_f,change%_livecattle_f
0,2019년 12월 31일,124.700,124.400,124.750,124.400,0.34K,0.48%
1,2019년 12월 30일,124.100,123.650,124.125,123.575,0.79K,0.49%
2,2019년 12월 27일,123.500,123.300,123.900,123.200,1.01K,0.18%
3,2019년 12월 26일,123.275,122.650,123.275,122.550,1.04K,-1.99%
4,2019년 12월 25일,125.775,125.775,125.775,125.775,-,2.55%
5,2019년 12월 24일,122.650,122.450,122.725,122.250,0.54K,0.20%
6,2019년 12월 23일,122.400,122.000,122.800,121.750,0.71K,0.14%
7,2019년 12월 20일,122.225,122.225,122.400,121.800,1.52K,0.08%
8,2019년 12월 19일,122.125,122.150,122.725,121.925,2.09K,-0.10%
9,2019년 12월 18일,122.250,122.300,122.650,121.825,1.66K,0.02%


In [26]:
data1.Date = data1.Date.str.replace('년 ','-')
data1.Date = data1.Date.str.replace('월 ','-')
data1.Date = data1.Date.str.replace('일','')

In [27]:
for i in data1.columns[1:]:
   # print(i)
    data1[i]
    col_name = '{}'.format(i)
    data1[i] = data1[i].astype('str')
    data1[i] = data1[i].str.replace('K','')
    data1[i] = data1[i].str.replace('%','')

data1.replace('-',np.nan, inplace=True)

In [28]:
data1.sort_values(by='Date', ascending=True, inplace=True)

# bdi

In [29]:
livestock_preprocesser.get_filelist('feature_data')[0]

'bdi.xlsx'

In [30]:
data_name = livestock_preprocesser.get_filelist('feature_data')[0]
data_name

'bdi.xlsx'

In [31]:
data2 = pd.read_excel("{}".format(data_name), thousands=',')

In [32]:
data2.head(10)

,2011-01-03 00:00:00,Unnamed: 1
0,2011-01-04,1693.0
1,2011-01-05,1621.0
2,2011-01-06,1544.0
3,2011-01-07,1519.0
4,2011-01-10,1495.0
5,2011-01-11,1480.0
6,2011-01-12,1453.0
7,2011-01-13,1438.0
8,2011-01-14,1439.0
9,2011-01-17,1439.0


In [33]:
col_list = ['Date','BDI']
data2.columns = col_list

In [34]:
data2 = livestock_preprocesser.set_funda_data(data2 , '2011-01-11')

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [35]:
data2.isnull().sum()

BDI    73
dtype: int64

In [36]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2209 entries, 2011-01-11 to 2019-12-30
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BDI     2136 non-null   float64
dtypes: float64(1)
memory usage: 34.5 KB


In [37]:
data2 = data2.reset_index()
data2.replace('-',np.nan, inplace=True)

In [38]:
data2.head()

,Date,BDI
0,2011-01-11,1480.0
1,2011-01-12,1453.0
2,2011-01-13,1438.0
3,2011-01-14,1439.0
4,2011-01-17,1439.0


# Kospi Volatility

In [39]:
data_name = livestock_preprocesser.get_filelist('feature_data')[5]
data_name

'kospi_volatility.csv'

In [40]:
data3 = pd.read_csv("{}".format(data_name), thousands=',')

In [41]:
data3.head()

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 30일,14.69,14.86,15.00,14.69,-,3.96%
1,2019년 12월 27일,14.13,14.36,14.72,14.13,-,-0.14%
2,2019년 12월 26일,14.15,14.11,14.20,14.03,-,2.46%
3,2019년 12월 24일,13.81,14.16,14.23,13.81,-,-1.15%
4,2019년 12월 23일,13.97,13.57,14.05,13.57,-,6.80%


In [42]:
# v-kospi
col_list = ['Date','close','open','high','low','volume','change%']
data3.colums = col_list

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name[:dotpos]
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

In [44]:
data3.columns = add_feature_name(col_list,data_name)
data3.head()

,Date,close_kospi_volatility,open_kospi_volatility,high_kospi_volatility,low_kospi_volatility,volume_kospi_volatility,change%_kospi_volatility
0,2019년 12월 30일,14.69,14.86,15.00,14.69,-,3.96%
1,2019년 12월 27일,14.13,14.36,14.72,14.13,-,-0.14%
2,2019년 12월 26일,14.15,14.11,14.20,14.03,-,2.46%
3,2019년 12월 24일,13.81,14.16,14.23,13.81,-,-1.15%
4,2019년 12월 23일,13.97,13.57,14.05,13.57,-,6.80%


In [45]:
data3.Date = data3.Date.str.replace('년 ','-')
data3.Date = data3.Date.str.replace('월 ','-')
data3.Date = data3.Date.str.replace('일','')

In [46]:
for i in data3.columns[1:]:
   # print(i)
    data3[i]
    col_name = '{}'.format(i)
    data3[i] = data3[i].astype('str')
    data3[i] = data3[i].str.replace('K','')
    data3[i] = data3[i].str.replace('%','')

data3.replace('-',np.nan, inplace=True)

In [47]:
data3.sort_values(by='Date', ascending=True, inplace=True)

In [48]:
data3.head(10)

,Date,close_kospi_volatility,open_kospi_volatility,high_kospi_volatility,low_kospi_volatility,volume_kospi_volatility,change%_kospi_volatility
1569,2013-08-06,14.98,14.79,14.98,14.61,NaN,0.40
1568,2013-08-07,15.35,15.02,15.42,14.8,NaN,2.47
1567,2013-08-08,14.96,14.84,15.42,14.77,NaN,-2.54
1566,2013-08-09,14.38,14.95,15.05,14.38,NaN,-3.88
1565,2013-08-12,14.72,14.84,15.22,14.64,NaN,2.36
1564,2013-08-13,14.26,14.74,14.78,14.26,NaN,-3.13
1563,2013-08-14,13.78,14.33,14.33,13.76,NaN,-3.37
1562,2013-08-16,14.23,14.68,14.75,14.23,NaN,3.27
1561,2013-08-19,14.51,14.88,14.88,14.51,NaN,1.97
1560,2013-08-20,15.58,14.89,15.71,14.67,NaN,7.37


In [49]:
data3['volume_kospi_volatility'].isnull().sum()

1570

In [50]:
data3.drop(['volume_kospi_volatility'],axis=1,inplace=True)

# 'South Korea 5-Year Bond Yield

In [51]:
data_name = livestock_preprocesser.get_filelist('feature_data')[2]
data_name

'South Korea 5-Year Bond Yield.csv'

In [52]:
data4 = pd.read_csv("{}".format(data_name), thousands=',')
data4.head(10)

,Date,Price,Open,High,Low,Change %
0,"Dec 31, 2019",1.470,1.470,1.471,1.470,-0.74%
1,"Dec 30, 2019",1.481,1.472,1.487,1.472,-0.27%
2,"Dec 29, 2019",1.485,1.485,1.485,1.485,0.00%
3,"Dec 28, 2019",1.485,1.485,1.485,1.485,0.00%
4,"Dec 27, 2019",1.485,1.496,1.496,1.480,1.02%
5,"Dec 26, 2019",1.470,1.470,1.474,1.455,0.68%
6,"Dec 25, 2019",1.460,1.460,1.460,1.460,0.62%
7,"Dec 24, 2019",1.451,1.465,1.465,1.449,-0.82%
8,"Dec 23, 2019",1.463,1.468,1.468,1.457,-0.81%
9,"Dec 22, 2019",1.475,1.475,1.475,1.475,0.00%


In [53]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name[:dotpos]
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

In [54]:
data4.columns = add_feature_name(data4.columns,data_name)

In [55]:
data4.head()

,Date,Price_South Korea 5-Year Bond Yield,Open_South Korea 5-Year Bond Yield,High_South Korea 5-Year Bond Yield,Low_South Korea 5-Year Bond Yield,Change %_South Korea 5-Year Bond Yield
0,"Dec 31, 2019",1.470,1.470,1.471,1.470,-0.74%
1,"Dec 30, 2019",1.481,1.472,1.487,1.472,-0.27%
2,"Dec 29, 2019",1.485,1.485,1.485,1.485,0.00%
3,"Dec 28, 2019",1.485,1.485,1.485,1.485,0.00%
4,"Dec 27, 2019",1.485,1.496,1.496,1.480,1.02%


In [56]:
data4.Date = pd.to_datetime(data4.Date)

In [57]:
data4.Date = data4.Date.astype('str')

In [58]:
data4.Date = data4.Date.str.replace('년 ','-')
data4.Date = data4.Date.str.replace('월 ','-')
data4.Date = data4.Date.str.replace('일','')

In [59]:
for i in data4.columns[1:]:
   # print(i)
    data4[i]
    col_name = '{}'.format(i)
    data4[i] = data4[i].astype('str')
    data4[i] = data4[i].str.replace('K','')
    data4[i] = data4[i].str.replace('%','')

data4.replace('-',np.nan, inplace=True)

In [60]:
data4.sort_values(by='Date', ascending=True, inplace=True)

In [61]:
data4.head(10)

,Date,Price_South Korea 5-Year Bond Yield,Open_South Korea 5-Year Bond Yield,High_South Korea 5-Year Bond Yield,Low_South Korea 5-Year Bond Yield,Change %_South Korea 5-Year Bond Yield
2977,2010-01-04,4.96,4.96,4.96,4.96,1.12
2976,2010-01-05,4.949,4.949,4.949,4.949,-0.22
2975,2010-01-06,4.88,4.88,4.88,4.88,-1.39
2974,2010-01-07,4.92,4.92,4.92,4.92,0.82
2973,2010-01-08,4.91,4.91,4.91,4.91,-0.20
2972,2010-01-11,4.84,4.84,4.84,4.84,-1.43
2971,2010-01-12,4.86,4.86,4.86,4.86,0.41
2970,2010-01-13,4.82,4.82,4.82,4.82,-0.82
2969,2010-01-14,4.775,4.775,4.775,4.775,-0.93
2968,2010-01-15,4.775,4.775,4.775,4.775,0.00


# USD_KRW

In [62]:
data_name = livestock_preprocesser.get_filelist('feature_data')[3]
data_name

'USD_KRW.csv'

In [63]:
data5 = pd.read_csv("{}".format(data_name), thousands=',')
data5.head(10)

,날짜,종가,오픈,고가,저가,변동 %
0,2019년 12월 31일,1155.07,1157.97,1159.19,1153.14,-0.25%
1,2019년 12월 30일,1157.97,1160.97,1161.30,1154.89,-0.15%
2,2019년 12월 27일,1159.68,1162.15,1162.83,1158.12,-0.23%
3,2019년 12월 26일,1162.30,1161.04,1163.42,1160.34,0.13%
4,2019년 12월 25일,1160.76,1163.52,1164.09,1160.68,-0.24%
5,2019년 12월 24일,1163.52,1164.04,1166.61,1160.71,-0.04%
6,2019년 12월 23일,1164.04,1159.39,1165.20,1159.38,0.40%
7,2019년 12월 20일,1159.40,1163.23,1166.05,1157.87,-0.33%
8,2019년 12월 19일,1163.27,1167.62,1168.24,1163.03,-0.29%
9,2019년 12월 18일,1166.66,1163.24,1168.89,1161.99,0.22%


In [64]:
# usd_krw
col_list = ['Date','close','open','high','low','change%']
data5.columns = col_list

In [65]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name[:dotpos]
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

In [66]:
data5.columns = add_feature_name(col_list,data_name)
data5.head()

,Date,close_USD_KRW,open_USD_KRW,high_USD_KRW,low_USD_KRW,change%_USD_KRW
0,2019년 12월 31일,1155.07,1157.97,1159.19,1153.14,-0.25%
1,2019년 12월 30일,1157.97,1160.97,1161.30,1154.89,-0.15%
2,2019년 12월 27일,1159.68,1162.15,1162.83,1158.12,-0.23%
3,2019년 12월 26일,1162.30,1161.04,1163.42,1160.34,0.13%
4,2019년 12월 25일,1160.76,1163.52,1164.09,1160.68,-0.24%


In [67]:
data5.Date = data5.Date.str.replace('년 ','-')
data5.Date = data5.Date.str.replace('월 ','-')
data5.Date = data5.Date.str.replace('일','')

In [68]:
for i in data5.columns[1:]:
   # print(i)
    data5[i]
    col_name = '{}'.format(i)
    data5[i] = data5[i].astype('str')
    data5[i] = data5[i].str.replace('K','')
    data5[i] = data5[i].str.replace('%','')

data5.replace('-',np.nan, inplace=True)

In [69]:
data5.head()

,Date,close_USD_KRW,open_USD_KRW,high_USD_KRW,low_USD_KRW,change%_USD_KRW
0,2019-12-31,1155.07,1157.97,1159.19,1153.14,-0.25
1,2019-12-30,1157.97,1160.97,1161.3,1154.89,-0.15
2,2019-12-27,1159.68,1162.15,1162.83,1158.12,-0.23
3,2019-12-26,1162.3,1161.04,1163.42,1160.34,0.13
4,2019-12-25,1160.76,1163.52,1164.09,1160.68,-0.24


In [70]:
data5.sort_values(by='Date', ascending=True, inplace=True)

In [71]:
data5.isnull().sum()

Date               0
close_USD_KRW      0
open_USD_KRW       0
high_USD_KRW       0
low_USD_KRW        0
change%_USD_KRW    0
dtype: int64

# grain
데이터 다시 구하기

In [72]:
# data_name = livestock_preprocesser.get_filelist('feature_data')[6]
# data_name

In [73]:
# data = pd.read_excel("{}".format(data_name), thousands=',')
# data.head(20)

In [74]:
# # grain
# col_list = ['Date','oats','corn','soybeanmeal','soybean']
# data.columns = col_list


In [75]:
# data.head(15)

In [76]:
# data = livestock_preprocesser.set_funda_data(data , '2011-01-11')

In [77]:
# data.reset_index(inplace=True)

In [78]:
# data.tail()

# lean_hog

In [79]:
data_name = livestock_preprocesser.get_filelist('feature_data')[4]
data_name

'leanhog_f.csv'

In [80]:
data7 = pd.read_csv("{}".format(data_name), thousands=',')
data7.head(20)

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2019년 12월 31일,71.42,71.70,71.88,70.83,12.23K,-0.53%
1,2019년 12월 30일,71.80,71.25,71.90,70.60,18.76K,1.73%
2,2019년 12월 27일,70.58,70.75,71.03,69.83,12.70K,-0.45%
3,2019년 12월 26일,70.90,71.05,71.50,70.05,9.91K,0.14%
4,2019년 12월 25일,70.80,70.80,70.80,70.80,-,0.14%
5,2019년 12월 24일,70.70,70.12,70.92,69.62,8.95K,0.64%
6,2019년 12월 23일,70.25,70.67,71.30,69.47,17.34K,-0.59%
7,2019년 12월 20일,70.67,70.85,71.22,70.30,15.25K,-0.42%
8,2019년 12월 19일,70.97,69.90,71.00,69.53,18.53K,1.53%
9,2019년 12월 18일,69.90,69.67,70.40,69.33,16.05K,0.07%


In [81]:
# lean_hog
col_list = ['Date','close','open','high','low','volume','change%']
data7.columns = col_list

In [82]:
def add_feature_name(feature_name_list, file_name):
    dotpos = file_name.find('.')
    name = file_name[:dotpos]
    new_col = ['Date']
    for i in feature_name_list[1:]:
        new_name = i + '_' +name
        new_col.append(new_name)
    return new_col

In [83]:
data7.columns = add_feature_name(col_list,data_name)
data7.head()

,Date,close_leanhog_f,open_leanhog_f,high_leanhog_f,low_leanhog_f,volume_leanhog_f,change%_leanhog_f
0,2019년 12월 31일,71.42,71.70,71.88,70.83,12.23K,-0.53%
1,2019년 12월 30일,71.80,71.25,71.90,70.60,18.76K,1.73%
2,2019년 12월 27일,70.58,70.75,71.03,69.83,12.70K,-0.45%
3,2019년 12월 26일,70.90,71.05,71.50,70.05,9.91K,0.14%
4,2019년 12월 25일,70.80,70.80,70.80,70.80,-,0.14%


In [84]:
data7.Date = data7.Date.str.replace('년 ','-')
data7.Date = data7.Date.str.replace('월 ','-')
data7.Date = data7.Date.str.replace('일','')

In [85]:
for i in data7.columns[1:]:
   # print(i)
    data7[i]
    col_name = '{}'.format(i)
    data7[i] = data7[i].astype('str')
    data7[i] = data7[i].str.replace('K','')
    data7[i] = data7[i].str.replace('%','')

data7.replace('-',np.nan, inplace=True)

In [86]:
data7.sort_values(by='Date', ascending=True, inplace=True)

In [87]:
data7.head()

,Date,close_leanhog_f,open_leanhog_f,high_leanhog_f,low_leanhog_f,volume_leanhog_f,change%_leanhog_f
2548,2010-01-01,65.51,65.51,65.51,65.51,NaN,-0.13
2547,2010-01-04,65.85,65.92,66.45,65.5,11.09,0.52
2546,2010-01-05,67.6,66.12,67.8,65.7,20.27,2.66
2545,2010-01-06,66.58,67.53,67.88,66.35,15.05,-1.51
2544,2010-01-07,67.1,66.83,68.1,66.83,16.86,0.78


In [88]:
data7.isnull().sum()

Date                  0
close_leanhog_f       0
open_leanhog_f        0
high_leanhog_f        0
low_leanhog_f         0
volume_leanhog_f     41
change%_leanhog_f     0
dtype: int64

# tech

In [89]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\축산업')

In [90]:
data_name ='event_embbed_with_tech_33_{}.xlsx'.format(company)
data_name

'event_embbed_with_tech_33_이지바이오.xlsx'

In [91]:
data8 = pd.read_excel("{}".format(data_name), thousands=',', index_col=0)
data8.tail(20)

,date,rsi_14,macd,cci,adx,stoch_slowk,stoch_slowd,willr,momentum,roc,...,event7,event8,event9,event10,event11,event12,event13,event14,event15,event16
2699,2019-12-02,32.415352,-73.775018,-153.676865,18.291309,7.273769,17.993552,-96.688742,-620.0,-11.211573,...,0.402649,0.130327,-0.122853,-0.182238,0.379171,0.198072,0.467477,0.200951,0.138492,0.236561
2700,2019-12-03,31.959836,-92.917327,-171.114443,20.102396,8.816978,10.206887,-86.627907,-525.0,-9.686347,...,0.276453,-0.072868,-0.363934,-0.410903,0.308101,0.020696,0.373200,0.037193,-0.051068,0.046044
2701,2019-12-04,30.720101,-110.046858,-144.950591,21.680451,14.372534,10.154427,-91.279070,-365.0,-6.992337,...,0.472049,0.150315,-0.142132,-0.206470,0.444258,0.241443,0.504278,0.225557,0.165089,0.255228
2702,2019-12-05,28.488675,-128.196242,-136.972388,23.358830,13.395062,12.194858,-98.850575,-370.0,-7.184466,...,0.489864,0.188518,-0.104852,-0.158550,0.439291,0.264467,0.528775,0.236874,0.201541,0.292932
2703,2019-12-06,35.242831,-132.977710,-105.373424,25.002469,18.008008,15.258535,-83.750000,-270.0,-5.242718,...,0.149202,-0.180185,-0.418890,-0.449263,0.153277,-0.101791,0.243045,-0.108771,-0.142401,-0.074144
2704,2019-12-09,34.887971,-136.006181,-75.648546,26.005444,34.674675,22.025915,-82.857143,-450.0,-8.458647,...,0.377846,0.055614,-0.126873,-0.191015,0.363734,0.129308,0.456418,0.105349,0.093219,0.111255
2705,2019-12-10,34.887971,-136.828986,-65.376641,26.936778,56.156156,36.279613,-80.645161,-420.0,-7.939509,...,0.391942,0.085678,-0.197395,-0.232123,0.431134,0.170702,0.463553,0.143189,0.116733,0.172978
2706,2019-12-11,36.373975,-134.318887,-53.760000,27.801588,70.370370,53.733734,-77.419355,-290.0,-5.598456,...,0.304656,-0.002774,-0.258106,-0.293796,0.330595,0.086295,0.431483,0.096564,0.019448,0.101045
2707,2019-12-12,37.900252,-129.226145,-30.899830,27.724472,70.843184,65.789903,-74.193548,-260.0,-5.029014,...,0.343200,-0.028182,-0.286659,-0.324200,0.271939,0.073483,0.388645,0.053686,-0.000064,0.057041
2708,2019-12-13,42.366850,-118.977117,4.611097,26.813165,68.910816,70.041457,-64.516129,-130.0,-2.549020,...,0.276853,0.049618,-0.170087,-0.243921,0.352473,0.163449,0.447540,0.144587,0.084767,0.182786


In [92]:
data8.columns

Index(['date', 'rsi_14', 'macd', 'cci', 'adx', 'stoch_slowk', 'stoch_slowd',
       'willr', 'momentum', 'roc', 'ema20', 'adosc', 'obv', 'event1', 'event2',
       'event3', 'event4', 'event5', 'event6', 'event7', 'event8', 'event9',
       'event10', 'event11', 'event12', 'event13', 'event14', 'event15',
       'event16'],
      dtype='object')

In [93]:
data8.columns = ['Date', 'rsi_14', 'macd', 'cci', 'adx', 'stoch_slowk', 'stoch_slowd',
       'willr', 'momentum', 'roc', 'ema20', 'adosc', 'obv', 'event1', 'event2',
       'event3', 'event4', 'event5', 'event6', 'event7', 'event8', 'event9',
       'event10', 'event11', 'event12', 'event13', 'event14', 'event15',
       'event16']

# event

In [94]:
# os.chdir(r'G:\공유 드라이브\Boad ADV Stock\For_Friday')

In [95]:
# data_name ='event_easybio.xlsx'

In [96]:
# data9 = pd.read_excel("{}".format(data_name), thousands=',')
# data9.head(20)

In [97]:
# data9.columns = ['Date', 'event1', 'event2', 'event3', 'event4', 'event5', 'event6',
#        'event7', 'event8', 'event9', 'event10', 'event11', 'event12',
#        'event13', 'event14', 'event15', 'event16']

# 시가 종가 profit

In [98]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\축산업\tech_data')

tech_data = pd.read_excel('{}_tech.xlsx'.format(company))
tech_data = tech_data[5:]
tech_data.columns = ['Date','open','high','low','last','volumne']
tech_data['profit_rate'] = (tech_data['last'] - tech_data['open'])/tech_data['open']*100
tech_data.head()

,Date,open,high,low,last,volumne,profit_rate
5,2009-01-02 00:00:00,1745,1855,1700,1750,6502103,0.286533
6,2009-01-05 00:00:00,1780,1865,1710,1865,5151387,4.77528
7,2009-01-06 00:00:00,1910,2045,1845,1900,9986364,-0.52356
8,2009-01-07 00:00:00,1910,2185,1870,2185,6531056,14.3979
9,2009-01-08 00:00:00,2150,2175,2005,2020,7831444,-6.04651


In [99]:
# tech_data.profit_rate = tech_data.profit_rate.shift(-1)
tech_data.Date = pd.to_datetime(tech_data.Date)
tech_data.head()

,Date,open,high,low,last,volumne,profit_rate
5,2009-01-02,1745,1855,1700,1750,6502103,0.286533
6,2009-01-05,1780,1865,1710,1865,5151387,4.77528
7,2009-01-06,1910,2045,1845,1900,9986364,-0.52356
8,2009-01-07,1910,2185,1870,2185,6531056,14.3979
9,2009-01-08,2150,2175,2005,2020,7831444,-6.04651


In [100]:
tech_data.tail()

,Date,open,high,low,last,volumne,profit_rate
2719,2019-12-23,4780,4875,4780,4845,374778,1.35983
2720,2019-12-24,4825,4850,4800,4825,205932,0
2721,2019-12-26,4825,4915,4810,4915,305143,1.86528
2722,2019-12-27,4875,4930,4860,4930,248591,1.12821
2723,2019-12-30,4930,4990,4905,4985,332172,1.11562


In [101]:
profit_rate = tech_data[['Date','profit_rate']]

# data concat

In [102]:
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,profit_rate,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),adjusted_BPS,adjusted_EPS
0,2010-01-11,NaN,NaN,NaN,2450,-3.73,2054718,1186071,NaN,NaN,NaN,NaN,NaN
1,2010-01-12,NaN,NaN,NaN,2640,7.76,2443731,2551680,NaN,NaN,NaN,NaN,NaN
2,2010-01-13,NaN,NaN,NaN,2575,-2.46,1879920,2561843,NaN,NaN,NaN,NaN,NaN
3,2010-01-14,NaN,NaN,NaN,2545,-1.17,1670983,1032237,NaN,NaN,NaN,NaN,NaN
4,2010-01-15,NaN,NaN,NaN,2660,4.52,2226197,3799154,NaN,NaN,NaN,NaN,NaN


In [103]:
data.drop('profit_rate',axis=1,inplace=True)

In [104]:
# df_list = [data,data1,data2,data3,data4,data5,data7,data8,data9,profit_rate]
df_list = [data,data1,data2,data3,data4,data5,data7,data8,profit_rate]

In [105]:
data.loc[:,'Date'] = pd.to_datetime(data.loc[:,'Date'])

In [106]:
for list_data in df_list:
    list_data.loc[:,'Date'] = pd.to_datetime(list_data.loc[:,'Date'])
#     list_data.Date = pd.to_datetime(list_data.Date)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [107]:
for feature_data in df_list[1:]:
    data = pd.merge(data,feature_data, on ='Date',how='left')

In [108]:
data.shape

(2461, 69)

In [109]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2460
Data columns (total 69 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Date                                    2461 non-null   datetime64[ns]
 1   asset(1000)                             1908 non-null   object        
 2   debt(1000)                              1908 non-null   object        
 3   sales(1000)                             2095 non-null   object        
 4   adjusted_price                          2461 non-null   object        
 5   volumne(mean_5d)(week)                  2461 non-null   object        
 6   volumne(week)                           2461 non-null   object        
 7   profit(1000)                            2095 non-null   object        
 8   term_profit(1000)                       2095 non-null   object        
 9   gross_margin(1000)                      2095 non-nul

In [110]:
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),...,event8,event9,event10,event11,event12,event13,event14,event15,event16,profit_rate
0,2010-01-11,NaN,NaN,NaN,2450,2054718,1186071,NaN,NaN,NaN,...,0.116433,-0.126272,-0.166290,0.427794,0.218076,0.507619,0.197948,0.160885,0.197399,-3.73281
1,2010-01-12,NaN,NaN,NaN,2640,2443731,2551680,NaN,NaN,NaN,...,-0.233234,-0.474628,-0.504328,0.185291,-0.152614,0.182645,-0.161057,-0.173912,-0.132418,7.09939
2,2010-01-13,NaN,NaN,NaN,2575,1879920,2561843,NaN,NaN,NaN,...,0.121049,-0.113419,-0.139299,0.419918,0.205617,0.490270,0.187930,0.143885,0.192334,-3.55805
3,2010-01-14,NaN,NaN,NaN,2545,1670983,1032237,NaN,NaN,NaN,...,0.192463,-0.028154,-0.075190,0.464002,0.281835,0.535903,0.267743,0.199049,0.295549,-1.73745
4,2010-01-15,NaN,NaN,NaN,2660,2226197,3799154,NaN,NaN,NaN,...,0.068302,-0.236157,-0.270052,0.405716,0.163633,0.453250,0.136207,0.075283,0.197806,2.30769


In [111]:
data.iloc[:,2:] = data.iloc[:,2:].astype('float64')

In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2460
Data columns (total 69 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Date                                    2461 non-null   datetime64[ns]
 1   asset(1000)                             1908 non-null   object        
 2   debt(1000)                              1908 non-null   float64       
 3   sales(1000)                             2095 non-null   float64       
 4   adjusted_price                          2461 non-null   float64       
 5   volumne(mean_5d)(week)                  2461 non-null   float64       
 6   volumne(week)                           2461 non-null   float64       
 7   profit(1000)                            2095 non-null   float64       
 8   term_profit(1000)                       2095 non-null   float64       
 9   gross_margin(1000)                      2095 non-nul

In [113]:
data.head()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),...,event8,event9,event10,event11,event12,event13,event14,event15,event16,profit_rate
0,2010-01-11,NaN,NaN,NaN,2450.0,2054718.0,1186071.0,NaN,NaN,NaN,...,0.116433,-0.126272,-0.166290,0.427794,0.218076,0.507619,0.197948,0.160885,0.197399,-3.732809
1,2010-01-12,NaN,NaN,NaN,2640.0,2443731.0,2551680.0,NaN,NaN,NaN,...,-0.233234,-0.474628,-0.504328,0.185291,-0.152614,0.182645,-0.161057,-0.173912,-0.132418,7.099391
2,2010-01-13,NaN,NaN,NaN,2575.0,1879920.0,2561843.0,NaN,NaN,NaN,...,0.121049,-0.113419,-0.139299,0.419918,0.205617,0.490270,0.187930,0.143885,0.192334,-3.558052
3,2010-01-14,NaN,NaN,NaN,2545.0,1670983.0,1032237.0,NaN,NaN,NaN,...,0.192463,-0.028154,-0.075190,0.464002,0.281835,0.535903,0.267743,0.199049,0.295549,-1.737452
4,2010-01-15,NaN,NaN,NaN,2660.0,2226197.0,3799154.0,NaN,NaN,NaN,...,0.068302,-0.236157,-0.270052,0.405716,0.163633,0.453250,0.136207,0.075283,0.197806,2.307692


In [114]:
data

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),...,event8,event9,event10,event11,event12,event13,event14,event15,event16,profit_rate
0,2010-01-11,NaN,NaN,NaN,2450.0,2054718.0,1186071.0,NaN,NaN,NaN,...,0.116433,-0.126272,-0.166290,0.427794,0.218076,0.507619,0.197948,0.160885,0.197399,-3.732809
1,2010-01-12,NaN,NaN,NaN,2640.0,2443731.0,2551680.0,NaN,NaN,NaN,...,-0.233234,-0.474628,-0.504328,0.185291,-0.152614,0.182645,-0.161057,-0.173912,-0.132418,7.099391
2,2010-01-13,NaN,NaN,NaN,2575.0,1879920.0,2561843.0,NaN,NaN,NaN,...,0.121049,-0.113419,-0.139299,0.419918,0.205617,0.490270,0.187930,0.143885,0.192334,-3.558052
3,2010-01-14,NaN,NaN,NaN,2545.0,1670983.0,1032237.0,NaN,NaN,NaN,...,0.192463,-0.028154,-0.075190,0.464002,0.281835,0.535903,0.267743,0.199049,0.295549,-1.737452
4,2010-01-15,NaN,NaN,NaN,2660.0,2226197.0,3799154.0,NaN,NaN,NaN,...,0.068302,-0.236157,-0.270052,0.405716,0.163633,0.453250,0.136207,0.075283,0.197806,2.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,2019-12-23,1.7848e+09,1.053581e+09,4.231081e+08,4845.0,443096.0,374778.0,-29040721.18,-36280723.37,30028406.75,...,-0.029674,-0.292408,-0.348478,0.285888,0.053829,0.377657,0.079178,-0.010560,0.094127,1.359833
2457,2019-12-24,1.7848e+09,1.053581e+09,4.231081e+08,4825.0,286344.0,205932.0,-29040721.18,-36280723.37,30028406.75,...,0.123012,-0.149042,-0.164908,0.386552,0.208794,0.538908,0.195410,0.175918,0.170087,0.000000
2458,2019-12-26,1.7848e+09,1.053581e+09,4.231081e+08,4915.0,281495.0,305143.0,-29040721.18,-36280723.37,30028406.75,...,0.072383,-0.216427,-0.255360,0.349998,0.130297,0.436854,0.127335,0.072740,0.151123,1.865285
2459,2019-12-27,1.7848e+09,1.053581e+09,4.231081e+08,4930.0,274864.0,248591.0,-29040721.18,-36280723.37,30028406.75,...,-0.053003,-0.325372,-0.362935,0.300481,0.035719,0.368325,0.035695,-0.006830,0.029371,1.128205


In [115]:

data['rate_rolling_3'] = data.profit_rate.rolling(window=3).mean()
data['rate_rolling_5'] = data.profit_rate.rolling(window=5).mean()
data['rate_rolling_10'] = data.profit_rate.rolling(window=10).mean()
data['rate_rolling_30'] = data.profit_rate.rolling(window=30).mean()
data['rate_rolling_120'] = data.profit_rate.rolling(window=120).mean()
data['rate_rolling_150'] = data.profit_rate.rolling(window=150).mean()

data['profit_rate'] = data.profit_rate.shift(periods=-1)

In [116]:
data.tail()

,Date,asset(1000),debt(1000),sales(1000),adjusted_price,volumne(mean_5d)(week),volumne(week),profit(1000),term_profit(1000),gross_margin(1000),...,event14,event15,event16,profit_rate,rate_rolling_3,rate_rolling_5,rate_rolling_10,rate_rolling_30,rate_rolling_120,rate_rolling_150
2456,2019-12-23,1.7848e+09,1.053581e+09,4.231081e+08,4845.0,443096.0,374778.0,-29040721.18,-36280723.37,30028406.75,...,0.079178,-0.010560,0.094127,0.000000,0.593113,0.235247,-0.091552,-0.378027,-0.279053,-0.382239
2457,2019-12-24,1.7848e+09,1.053581e+09,4.231081e+08,4825.0,286344.0,205932.0,-29040721.18,-36280723.37,30028406.75,...,0.195410,0.175918,0.170087,1.865285,0.523232,0.589414,-0.132789,-0.439755,-0.284674,-0.382239
2458,2019-12-26,1.7848e+09,1.053581e+09,4.231081e+08,4915.0,281495.0,305143.0,-29040721.18,-36280723.37,30028406.75,...,0.127335,0.072740,0.151123,1.128205,1.075039,0.728925,-0.028734,-0.371540,-0.256546,-0.369804
2459,2019-12-27,1.7848e+09,1.053581e+09,4.231081e+08,4930.0,274864.0,248591.0,-29040721.18,-36280723.37,30028406.75,...,0.035695,-0.006830,0.029371,1.115619,0.997830,0.912637,0.114543,-0.315883,-0.245725,-0.358248
2460,2019-12-30,1.7848e+09,1.053581e+09,4.231081e+08,4985.0,293323.0,332172.0,-29040721.18,-36280723.37,30028406.75,...,0.144655,0.085198,0.154636,NaN,1.369703,1.093788,0.216034,-0.357916,-0.244960,-0.328554


In [117]:
os.chdir(r'G:\공유 드라이브\Boad ADV Stock\축산업')
data.to_csv("data_1_{}.csv".format(company))